In [ ]:
import os

import geopandas as gpd
import pandas as pd
pd.options.display.max_columns = None
import matplotlib
%matplotlib inline
import shapely.geometry


In [ ]:
cols = ["tpep_pickup_datetime", "tpep_dropoff_datetime", 
        "pickup_longitude", "pickup_latitude", 
        "dropoff_longitude", "dropoff_latitude",
        "trip_distance"]
boroughs = gpd.read_file("data/nybb_18a/nybb.shp")
manhattan = boroughs[boroughs.BoroName == "Manhattan"].to_crs(epsg=4326).reset_index(drop=True)

In [ ]:
def process_month(month):
    print "processing month {}".format(month)
    raw_fname = "data/taxi_raw/yellow_2016{}.csv".format("0" + str(month))
    df = pd.read_csv(raw_fname)[cols]
    print "raw df has {} rows".format(len(df))
    before_5am = df[df["tpep_pickup_datetime"].str.split(" ", 1).apply(lambda x: int(x[1].split(":")[0])) < 5]
    before_5am["pickup_point"] = before_5am[["pickup_longitude", "pickup_latitude"]].apply(shapely.geometry.Point, axis=1)
    before_5am["dropoff_point"] = before_5am[["dropoff_longitude", "dropoff_latitude"]].apply(shapely.geometry.Point, axis=1)
    before_5am = before_5am.reset_index(drop=True)
    print "before 5am has {} rows".format(len(before_5am))
    before_5am["geometry"] = before_5am["pickup_point"]
    before_5am = gpd.GeoDataFrame(before_5am)
    #before_5am = before_5am.set_geometry(before_5am["pickup_point"], crs={"epsg": 4326})
    before_filter_pickup = gpd.sjoin(before_5am, manhattan, how='inner', op='within')
    del before_filter_pickup["index_right"]
    before_filter_pickup["geometry"] = before_filter_pickup["dropoff_point"]
    before_filter_pickup.set_geometry("geometry", 
                                      crs={"epsg": 4326}, inplace=True)
    before_filter_dropoff = gpd.sjoin(before_filter_pickup, manhattan, how='inner', op='within')
    before_filter_dropoff = before_filter_dropoff[cols]
    before_filter_dropoff.to_pickle("data/taxi_clean/2016{}_filtered_cython.pkl".format("0" + str(month)))
    print "filtered df has {} rows".format(len(before_filter_dropoff))
    del before_filter_dropoff
    del before_filter_pickup
    del before_5am
    del df
    print "======"
    return "OK"

In [ ]:
for month in range(1, 6):
    process_month(month)

## Part 2: restart notebook before running this! 

## WARNING: Requires 13GB memory

In [3]:
nx.__version__

'2.1'

In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry
from shapely.ops import nearest_points
import networkx as nx

import glob
import numpy as np

cols = ["tpep_pickup_datetime", "tpep_dropoff_datetime", 
        "pickup_longitude", "pickup_latitude", 
        "dropoff_longitude", "dropoff_latitude",
        "trip_distance"]

node_cols = ["NODEID", "geometry"]

In [4]:
files = glob.glob("data/taxi_clean/*_filtered.pkl")
nodes = gpd.read_file("data/lion/lion.shp/node.shp").to_crs(epsg=4326)[node_cols]
def uniform_str(x):
    strd = str(x)
    while len(strd) < 7:
        strd = '0' + strd
    return strd

nodes["NODEID_STR"] = nodes["NODEID"].apply(uniform_str)
nodes.set_index("NODEID_STR", inplace=True)

g = set(nx.read_gpickle("data/mn_subgraph.pkl").nodes())
for x in g:
    if not x in nodes.index:
        print x
print len(nodes)
nodes = nodes.loc[g].reset_index()[node_cols]
print len(nodes)
del g

I am densified (external_values, 131335 elements)
131335
12263


In [5]:
i = 0
def closest_p(g):
    global i
    i = i + 1
    if i % 100000 == 0:
        print i
    if len(g) == 1 or i > 10:
        return g.iloc[0]
  
    s = g.pickup_point.iloc[0]
    nod = nodes.loc[g.index_right].geometry
    distances = [s.distance(s2) for s2 in nod]
    return g.iloc[np.argmin(distances)]

def closest_d(g):
    global i
    i = i + 1
    if i % 100000 == 0:
        print i
    if len(g) == 1 or i > 10:
        return g.iloc[0]
  
    s = g.dropoff_point.iloc[0]
    nod = nodes.loc[g.index_right].geometry
    distances = [s.distance(s2) for s2 in nod]
    return g.iloc[np.argmin(distances)]

def handle_file(f):
    global i 
    i = 0
    out = f.split(".")[0] + "_od_v2.pkl"
    p = pd.read_pickle(f)
    orig = len(p)
    p["pickup_point"] = p[["pickup_longitude", "pickup_latitude"]].apply(shapely.geometry.Point, axis=1)
    p["geometry"] = p["pickup_point"].apply(lambda x: x.buffer(.0005))
    p = gpd.GeoDataFrame(p)
    joined = gpd.sjoin(p, nodes, how='inner', op='contains')
    del p
    closest_only = joined.groupby(joined.index).apply(closest_p)
    closest_only["NODEID_O"] = closest_only["NODEID"]

    del joined
    del closest_only["NODEID"]
    del closest_only["pickup_point"]
    closest_only = closest_only[cols + ["NODEID_O"]]
    closest_only["dropoff_point"] = closest_only[["dropoff_longitude", "dropoff_latitude"]].apply(shapely.geometry.Point, axis=1)
    closest_only["geometry"] = closest_only["dropoff_point"].apply(lambda x: x.buffer(.0005))
    closest_only = gpd.GeoDataFrame(closest_only)
    joined = gpd.sjoin(closest_only, nodes, how='inner', op='contains')
    
    i = 0
    final = joined.groupby(joined.index).apply(closest_d)
    del joined
    del closest_only
    final["NODEID_D"] = final["NODEID"]
    final = final[cols + ["NODEID_O", "NODEID_D"]]
    final.to_pickle(out)
    
    print "orig had {}".format(orig)
    print "final had {}".format(len(final))
    print "{} %".format(orig / float(len(final)))
    return final

for f in files:
    print "handling {}".format(f)
    out_thing = handle_file(f)
    print "==="

handling data/taxi_clean/201602_filtered.pkl


/home/christian/.miniconda2/envs/subway_gpd_dev/lib/python2.7/site-packages/geopandas/tools/sjoin.py:53: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


['trip_distance', 'tpep_dropoff_datetime', 'geometry', u'NODEID', 'pickup_longitude', 'dropoff_longitude', 'index_right', 'pickup_point', 'pickup_latitude', 'dropoff_latitude', 'tpep_pickup_datetime']
100000
200000
300000
400000
500000
600000
700000
['trip_distance', 'tpep_dropoff_datetime', 'geometry', u'NODEID', 'pickup_longitude', 'NODEID_O', 'dropoff_longitude', 'dropoff_point', 'index_right', 'pickup_latitude', 'dropoff_latitude', 'tpep_pickup_datetime']
100000
200000
300000
400000
500000
orig had 821880
final had 566836
1.44994319345 %
===
handling data/taxi_clean/201605_filtered.pkl
['trip_distance', 'tpep_dropoff_datetime', 'geometry', u'NODEID', 'pickup_longitude', 'dropoff_longitude', 'index_right', 'pickup_point', 'pickup_latitude', 'dropoff_latitude', 'tpep_pickup_datetime']
100000
200000
300000
400000
500000
600000
700000
['trip_distance', 'tpep_dropoff_datetime', 'geometry', u'NODEID', 'pickup_longitude', 'NODEID_O', 'dropoff_longitude', 'dropoff_point', 'index_right', 'p

In [ ]:
!ls -lh data/taxi_clean/*_od.pkl

In [ ]:
joined = gpd.sjoin(p, nodes, how='inner', op='contains')

In [ ]:
del joined["VIntersect"]
del joined["index_righ"]
del joined["BoroCode"]
del joined["BoroName"]
del joined["Shape_Leng"]


In [ ]:

closest_only = joined.groupby(joined.index).apply(closest)

In [ ]:
print len(closest_only) / float(len(p))

In [ ]:
closest_only.head()

In [ ]:
# pts = nodes.geometry.unary_union
# i = 0
# def near(row, pts=pts):
#     global i
#     i = i + 1
#     if i % 100 == 0:
#         print "{} out of {}".format(i, len(p))
#     point = row.geometry
#     n = nearest_points(point, pts)
#     nearest = nodes.geometry == n[0]
#     return nodes[nearest].NODEID.get_values()[0]
# p["nearest_origin"] = p.apply(lambda row: near(row), axis=1)